In [44]:
import requests
import logging

In [12]:
def get_result_count(url, params):
    params['rows'] = 0
    
    r = requests.get(url, params=params)
    
    j = r.json()
    
    return int(j['message']['total-results'])

In [40]:
def get_dois(query, just_doi=False):
    base_url = "http://api.crossref.org/works"
    
    if just_doi:
        field = 'DOI'
    else:
        field = 'URL'
    
    doi_list = []
    
    # Set the params for the query
    params = {'filter': query}
    
    count = get_result_count(base_url, params)
    
    logging.debug('Total result count %d' % count)
    
    if count < 1000:
        # We can get them all in one go
        params['rows'] = 1000
        r = requests.get(base_url, params=params)
        
        doi_list = [i['URL'] for i in r.json()['message']['items']]
    else:
        # We need to use cursors etc
        
        # Call once with cursor set to *
        params['rows'] = 500
        
        params['cursor'] = '*'
        r = requests.get(base_url, params=params)
        
        logging.debug('Getting %s' % r.url)
        cursor = r.json()['message']['next-cursor']
        
        doi_list = [i['URL'] for i in r.json()['message']['items']]

        params['cursor'] = cursor
        
        while True:
            r = requests.get(base_url, params=params)
            
            logging.debug('Getting %s' % r.url)
            
            new_dois = [i['URL'] for i in r.json()['message']['items']]
            
            
            doi_list += new_dois
            
            logging.debug("Received %d items" % len(r.json()['message']['items']))

            if len(r.json()['message']['items']) < params['rows']:
                break
                
            cursor = r.json()['message']['next-cursor']
            params['cursor'] = cursor

        
    return doi_list

In [41]:
res = get_dois("type:journal-article,issn:1932-6203,from-pub-date:2015-01,until-pub-date:2015-02")

2303
http://api.crossref.org/works?filter=type%3Ajournal-article%2Cissn%3A1932-6203%2Cfrom-pub-date%3A2015-01%2Cuntil-pub-date%3A2015-02&cursor=%2A&rows=500
AoE/D2h0dHA6Ly9keC5kb2kub3JnLzEwLjEzNzEvam91cm5hbC5wb25lLjAxMTYzNDM=
500
http://dx.doi.org/10.1371/journal.pone.0096102
http://api.crossref.org/works?filter=type%3Ajournal-article%2Cissn%3A1932-6203%2Cfrom-pub-date%3A2015-01%2Cuntil-pub-date%3A2015-02&cursor=AoE%2FD2h0dHA6Ly9keC5kb2kub3JnLzEwLjEzNzEvam91cm5hbC5wb25lLjAxMTYzNDM%3D&rows=500
http://dx.doi.org/10.1371/journal.pone.0116344
500
http://api.crossref.org/works?filter=type%3Ajournal-article%2Cissn%3A1932-6203%2Cfrom-pub-date%3A2015-01%2Cuntil-pub-date%3A2015-02&cursor=AoE%2FD2h0dHA6Ly9keC5kb2kub3JnLzEwLjEzNzEvam91cm5hbC5wb25lLjAxMTY5OTI%3D&rows=500
http://dx.doi.org/10.1371/journal.pone.0116993
500
http://api.crossref.org/works?filter=type%3Ajournal-article%2Cissn%3A1932-6203%2Cfrom-pub-date%3A2015-01%2Cuntil-pub-date%3A2015-02&cursor=AoE%2FD2h0dHA6Ly9keC5kb2kub3JnLzEwLjEzNz